# TicTacToe

In this notebook we will focus on implementing this game in a more abstract setting than usual. To lay down the basics however, two dimensions will suffice. A further generalization will be implemented lateron.

## The Board

Establishing a digital representation is easily done with `numpy` and a Matrix $B \in \{ 0, \pm 1 \}^{n \times n}$. We identify "empty" with $0$ and $X \cong  1$, $O \cong -1$. The reason for using this identification becomes clear when doing calculations with board positions.

In [1]:
import numpy as np

Consider a player marking the position $p$ on the board $B$. To check, whether $p$ is a legal choice for a position, we check for two necessary conditions.

1. The position has not been marked before; i.e. $B_p = 0$
2. The position is actually on the board; i.e. $p \in \{ 0, \ldots, n-1 \}^2$

In [3]:
def is_legal_position(board, position):
    B = board
    p = position

    n = B.shape[0]

    if B[p] != 0:
        return False

    for p_ in p:
        if not p < n:
            return False

    return True

## The Players

TicTacToe is a game between two players, where the first to achive a TicTacToe wins the game. For reasons that will become clear lateron, it is useful to generalize by the following:

When a player makes a move, we must check for all possible ways a TicTacToe could occur. These will be summed up as points and added to the active player's `score`. When all board positions are marked, then the player with the highest score wins. If they match, then the player who `scored_first` wins. A draw occurs if nobody scored.

To store player information, we introduce the pyton class `player` and a dictionary `players` with two instances.

In [4]:
class player:
    def __init__(self, name):
        self.name  = name
        self.score = 0

In [7]:
players = {'X': player("Alice"),
           'O': player("Bob")}

for XO in {'X', 'O'}:
    print("id:   ", XO)
    print("name: ", players[XO].name)
    print("score:", players[XO].score)
    print("")

id:    X
name:  Alice
score: 0

id:    O
name:  Bob
score: 0



Let $p$ be, once again, the position on the board $B$, which gets marked by a player. We have to check if (and how many) TicTacToe have been achived by implementing a further method `update_score`. One way of doing so is enumerating every possible line $L \in \{ 0, \ldots, n-1 \}^{2 \times n}$, of lenght $n$, that passes through $p$ and checking $L$ for being TicTacToe.

1. By $n-1$ times increasing the first coordinate $p_0$ we obtain a horizontal line. This must be done modulo $n-1$. The same thing can be said about $p_1$ and a vertical line. If $p_0 = p_1$, then $p$ is on the main diagonal of $B$, and $p_0 = n - 1 - p_1$ accounts for the side diagnal.

In [135]:
def get_lines(board, point):
    B = board
    p = point

    x = p[0]
    y = p[1]
    n = B.shape[0]

    lines = []

    # vertical line
    lines += [[((x+i)%n, y) for i in range(n)]]

    # horizontal line
    lines += [[(x, (y+i)%n) for i in range(n)]]

    # main diagonal
    if x == y:
        lines += [[(i, i) for i in range(n)]]

    # side diagonal
    if x == n-1-y:
        lines += [[(n-1-i, i) for i in range(n)]]

    return lines

In [81]:
p = (1, 1)

print(get_lines(B, p))

[[(1, 1), (2, 1), (0, 1)], [(1, 1), (1, 2), (1, 0)], [(0, 0), (1, 1), (2, 2)], [(2, 0), (1, 1), (0, 2)]]


2. The fact that the matrix $B$ entries on the line $L$ form a "TicTacToe" is equivalent to

$$
| \sum_{i=0}^{n-1} A_{L_i} | = n.
$$.

In [134]:
def is_tictactoe(matrix, line):
    B = matrix
    L = line
    n = B.shape[0]

    S = np.array([B[L[i]] for i in range(n)])

    return abs(np.sum(S)) == n

In [117]:
B = np.identity(n)

print("B =")
display(B)

p = (1, 0)

lines = get_lines(B, p)

for L in lines:
    print("L =", L)
    print("Is L a TicTacToe? ...", is_tictactoe(B, L))
    print("")

B =


array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

L = [(1, 0), (2, 0), (0, 0)]
Is L a TicTacToe? ... False

L = [(1, 0), (1, 1), (1, 2)]
Is L a TicTacToe? ... False



Now, we can easily implement `update_score`. It shall be called, once a player marks a position $p$ on the board $B$.

In [150]:
class player:

    def __init__(self, XO, name):
        self.XO    = XO
        self.name  = name
        self.score = 0

    def update_score(self, board, position):
        B = board
        p = position

        lines = get_lines(B, p)

        for L in lines:
            if is_tictactoe(B, L):
                self.score += 1

In [153]:
XO   = 1
name = "Alice"

player_ = player(XO, name)

B = np.array([[1, -1,  0],
              [1,  1, -1],
              [0,  0, -1]])

# marks board position
p = (2, 0)
assert B[p] == 0
B[p] = player_.XO

print("B =")
display(B)

player_.update_score(B, p)

print(player_.name + "'s score:", player_.score)

B =


array([[ 1, -1,  0],
       [ 1,  1, -1],
       [ 1,  0, -1]])

Alice's score: 1


In order to determine, who won the game, we need to take into account the player's scores and who `scored_first`.

In [ ]:
def get_winner(players, scored_first):
    if players['X'].score < players['O'].score:
        return 'O'

    if players['O'].score < players['X'].score:
        return 'X'

    if players['X'].score == players['O'].score:
        return scored_first

## The Game

A nice looking user interface will not be implemented in this notebook. Please refer to the source code provided to find out more about that.

To beginn with, we have to take care of some simple initializations. What firstly comes to mind, is of course the board itself and the participating players. Furthermore, we'll track, who's `turn` it is, and who `scored_first`, by using `XO`.

---

Of course, $B$ starts out empty, but turns full iff

$$ \prod_{i,j=0}^{n-1} B_{i,j} \neq 0. $$

In [ ]:
n = input("Game dimension: ")

B = np.zeros((n, n))

name_X = input("Player X's name: ")
name_O = input("Player O's name: ")

players = {'X': player( 1, name_X),
           'O': player(-1, name_O)}

turn = 1
scored_first = ''

# game starts

while np.prod(B) == 0:
    XO = ''
    for XO_ in {'X', 'O'}
        if players[XO_].XO == turn:
            XO = XO_

    if scored_first == '' and players[XO].score != 0:
        scored_first = XO

    turn *= -1

# game ends

winner = get_winner(players, scored_first)

winner_name = ""
if winner == '':
    winner_name = "Nobody"
else:
    winner_name = players[winner].name

print(winner_name + " wins the game!")
print("")

for XO in {'X', 'O'}:
    print(players[XO].name + "'s score:'")
    print(players[XO].score)
    print("")